In [460]:
import cv2
import numpy as np
from cv2 import VideoWriter, VideoWriter_fourcc

In [461]:
frame = cv2.imread(filename='white_background.jpg')
car = cv2.imread(filename='car.png')
car = cv2.resize(car,(100,200))
bus = cv2.imread(filename='bus.png')
bus = cv2.resize(bus,(180,350))
car2 = cv2.imread(filename='car2.png')
car2 = cv2.resize(car2,(100,200))
x_offset_car = 450
y_offset_car = 400
x_offset_bus = 750
y_offset_bus = 200
x_offset_car2 = 150
y_offset_car2 = 400
frame[y_offset_car:y_offset_car+car.shape[0], x_offset_car:x_offset_car+car.shape[1]] = car
frame[y_offset_bus:y_offset_bus+bus.shape[0], x_offset_bus:x_offset_bus+bus.shape[1]] = bus
frame[y_offset_car2:y_offset_car2+car2.shape[0], x_offset_car2:x_offset_car2+car2.shape[1]] = car2
cv2.imwrite("Generated_frame.png", frame)

True

In [462]:
# Initialize video parameters
width = 1024
height = 600
FPS = 30
seconds = 5
initial_y_offset = 400
in_frame = True
boundary = 50

In [463]:
fourcc = VideoWriter_fourcc(*'MP42')
video = VideoWriter('./HUD.mp4', fourcc, float(FPS), (width, height))
background = cv2.imread(filename='white_background.jpg')
frame = background
for _ in range(FPS * seconds):
    frame.fill(255) # generate new white background
    car = car
    bus = bus
    car2 = car2
    if (y_offset_car > boundary and y_offset_bus > boundary and y_offset_car2 > boundary):
        y_offset_car -= 6
        frame[y_offset_car:y_offset_car+car.shape[0], x_offset_car:x_offset_car+car.shape[1]] = car
        if (in_frame):
            y_offset_bus -= 2
            y_offset_car2 -= 4
            frame[y_offset_bus:y_offset_bus+bus.shape[0], x_offset_bus:x_offset_bus+bus.shape[1]] = bus
            frame[y_offset_car2:y_offset_car2+car2.shape[0], x_offset_car2:x_offset_car2+car2.shape[1]] = car2
    elif (y_offset_car <= boundary):
        y_offset_car = initial_y_offset
        frame[y_offset_car:y_offset_car+car.shape[0], x_offset_car:x_offset_car+car.shape[1]] = car
        in_frame = False
    video.write(frame)
video.release()

In [475]:
'''
Utility class for file conversion.
'''
import imageio
import os, sys
imageio.plugins.ffmpeg.download()

class TargetFormat(object):
    GIF = ".gif"
    MP4 = ".mp4"
    AVI = ".avi"
    
def convert(input_path, target_format):
    output_path = os.path.splitext(input_path)[0] + target_format
    print("Converting\r\n\t{0}\r\nto\r\n\t{1}".format(input_path, output_path))
    reader = imageio.get_reader(input_path)
    fps = reader.get_meta_data()['fps']
    
    writer = imageio.get_writer(output_path, fps=fps)
    for i, frame in enumerate(reader):
        sys.stdout.write("\rframe {0}".format(i))
        sys.stdout.flush()
        writer.append_data(frame)
    print("\r\nFinalizing...")
    writer.close()
    print("Done.")

Imageio: 'ffmpeg-osx-v3.2.4' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-osx-v3.2.4 (33.8 MB)
Downloading: 8192/35458856 bytes (0.0114688/35458856 bytes (0.3360448/35458856 bytes (1.0507904/35458856 bytes (1.4761856/35458856 bytes (2.1770048/35458856 bytes (2.21097728/35458856 bytes (3.1%1179648/35458856 bytes (3.3%1474560/35458856 bytes (4.2%1703936/35458856 bytes (4.8%2260992/35458856 bytes (6.4%2818048/35458856 bytes (7.9%3399680/35458856 bytes (9.6%3735552/35458856 bytes (10.54210688/35458856 bytes (11.94562944/35458856 bytes (12.94603904/35458856 bytes (13.04669440/35458856 bytes (13.25423104/35458856 bytes (15.35701632/35458856 bytes (16.15832704/35458856 bytes (16.46045696/35458856 bytes (17.06537216/35458856 bytes (18.46733824/35458856 bytes (19.07143424/35458856 bytes (20.17241728/35458856 bytes (20.47798784/35458856 bytes (22.07864320/35458856 bytes (22.28093696/35458856 bytes (2

In [476]:
convert("./HUD.mp4", TargetFormat.GIF)

Converting
	./HUD.mp4
to
	./HUD.gif
frame 149
Finalizing...
Done.
